In [1]:
import time
plotter_start = time.time()

In [2]:
import uproot
import awkward as ak
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from matplotlib import colormaps
import mplhep as hep
hep.style.use("CMS")

In [3]:
def convertToHadronType(pdgIds):
    retIDs = []
    for pdgId in pdgIds:
        checkPdgId = np.abs(pdgId)
        if (checkPdgId == 521) or (checkPdgId == 511) or (checkPdgId == 531) or (checkPdgId == 541):
            retIDs.append(0) # B meson
        elif (checkPdgId == 5122) or (checkPdgId == 5112) or (checkPdgId == 5212) or (checkPdgId == 5222) or (checkPdgId == 5132) or (checkPdgId == 5232) or (checkPdgId == 5332) or (checkPdgId == 5142) or (checkPdgId == 5242) or (checkPdgId == 5342) or (checkPdgId == 5512) or (checkPdgId == 5532) or (checkPdgId == 5542) or (checkPdgId == 5554):
            retIDs.append(1) # B baryon
        elif (checkPdgId == 411) or (checkPdgId == 421) or (checkPdgId == 431):
            retIDs.append(2) # C meson
        elif (checkPdgId == 4122) or (checkPdgId == 4222) or (checkPdgId == 4212) or (checkPdgId == 4112) or (checkPdgId == 4232) or (checkPdgId == 4132) or (checkPdgId == 4332) or (checkPdgId == 4412) or (checkPdgId == 4422) or (checkPdgId == 4432) or (checkPdgId == 4444):
            retIDs.append(3) # C baryon
        elif (checkPdgId == 3122) or (checkPdgId == 3222) or (checkPdgId == 3212) or (checkPdgId == 3112) or (checkPdgId == 3322) or (checkPdgId == 3312) or (checkPdgId == 3334):
            retIDs.append(4) # S baryon
        else:
            print("You have a weird PDG ID. Probably go back and check the C++ code.")
            retIDs.append(-1) # error
    return np.array(retIDs)

In [4]:
def getBinomErr(eff, N):
    if N == 0: return 0.0
    up = eff * (1-eff)
    return np.sqrt(up / N)

In [5]:
def getMean(hist, binning):
    midpoints = (binning[1:] + binning[:-1]) / 2
    sum = 0.0
    for i in np.arange(len(hist)):
        sum += hist[i] * midpoints[i]
    return sum / np.sum(hist)

In [6]:
def getRmsErr(mean, hist, binning):
    midpoints = (binning[1:] + binning[:-1]) / 2
    rms = 0.0
    for i in np.arange(len(hist)):
        rms += hist[i] * (midpoints[i]-mean) * (midpoints[i]-mean)
    rms /= np.sum(hist)
    return np.sqrt(rms)

In [7]:
def plotVars(hists, binning, labels, xlabel, file_name, save_dir, log=False, uniform_bins=True, xticks=[]):
    plt.figure()
    midpoints = (binning[1:] + binning[:-1]) / 2
    xerr = (binning[1:] - binning[:-1]) / 2
    max_height = 0.0
    for i in np.arange(len(hists)):
        max_height = max(max_height, max(hists[i]))
        mean = getMean(hists[i], binning)
        rms = getRmsErr(mean, hists[i], binning)
        if rms < 10: label = labels[i] + (", mean=%.2f, RMS=%.2f" % (mean, rms))
        else:        label = labels[i] + (", mean=%.1f, RMS=%.1f" % (mean, rms))
        plt.hist(midpoints, binning, weights=hists[i], log=log, label=label, histtype="step", linewidth=2)
    plt.legend()
    if log: plt.ylim(top=max_height*300)
    else:   plt.ylim(top=max_height*1.4)
    plt.xlabel(xlabel)
    if uniform_bins: plt.ylabel("Events / %.3f" % (binning[1] - binning[0]))
    else:            plt.ylabel("Events")
    if len(xticks): plt.xticks(midpoints, xticks, rotation=45)
    hep.style.use("CMS")
    hep.cms.label("Private Work", data=False, rlabel="")
    plt.savefig(save_dir + file_name + ".pdf", bbox_inches="tight")
    plt.savefig(save_dir + file_name + ".png", bbox_inches="tight")
    plt.close()

In [8]:
def plotEffs(numerators, denominators, binning, labels, xlabel, file_name, save_dir, xticks=[]):
    midpoints = (binning[1:] + binning[:-1]) / 2
    xerr = (binning[1:] - binning[:-1]) / 2
    effs = []
    efferrs = []
    for i in np.arange(len(numerators)):
        eff = numerators[i] / denominators[i]
        effs.append(eff)
        efferrs.append(np.array([getBinomErr(eff[j], denominators[i][j]) for j in np.arange(len(eff))]))
    plt.figure()
    for i in np.arange(len(effs)):
        label = "%s, IE=%.3f" % (labels[i], (np.sum(numerators[i]) / np.sum(denominators[i])))
        plt.errorbar(midpoints, effs[i], efferrs[i], xerr, marker=".", linestyle="", markersize=15, label=label)
    plt.legend()
    plt.ylim(0.0, 0.8)
    plt.xlabel(xlabel)
    plt.ylabel("Efficiency")
    if len(xticks): plt.xticks(midpoints, xticks, rotation=45)
    hep.style.use("CMS")
    hep.cms.label("Private Work", data=False, rlabel="")
    plt.savefig(save_dir + file_name + ".pdf", bbox_inches="tight")
    plt.savefig(save_dir + file_name + ".png", bbox_inches="tight")
    plt.close()

In [9]:
# def plot2D(x, y, xbins, ybins, min, max, xlabel, ylabel, file_name, save_dir, log=True):
def plot2D(X, xbins, ybins, xlabel, ylabel, file_name, save_dir, log=True):
    xmidpoints = (xbins[1:] + xbins[:-1]) / 2
    ymidpoints = (ybins[1:] + ybins[:-1]) / 2
    xentries = []
    yentries = []
    weights = []
    for x in np.arange(len(xmidpoints)):
        for y in np.arange(len(ymidpoints)):
            for i in np.arange(int(X[x][y])):
                xentries.append(xmidpoints[x])
                yentries.append(ymidpoints[y])
    corr_coeff, _ = pearsonr(xentries, yentries)
    plt.figure(figsize=[10.0, 8.0])
    if log: plt.hist2d(xentries, yentries, bins=[xbins, ybins], norm="log", cmap="plasma")
    else:   plt.hist2d(xentries, yentries, bins=[xbins, ybins], cmap="plasma")
    plt.colorbar()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    ax = plt.gca()
    plt.text(0.65, 0.9, "R=%.3f" % corr_coeff, transform=ax.transAxes)
    hep.cms.label("Private Work", data=False, rlabel="")
    plt.savefig(save_dir + file_name + ".pdf", bbox_inches="tight")
    plt.savefig(save_dir + file_name + ".png", bbox_inches="tight")
    plt.close()

In [10]:
# gp_collections_ = [
#     "strange",
# ]
# gp_branches_ = [
#     "pt",
#     "eta",
#     "phi"
#     "pdgId",
#     "status",
#     "nDaughters",
#     "daughterPdgIds", # Special case
# ]

trk_collections = [
    "st",           # SimTrack
    "all",          # All reco tracks
    "match_gp",     # Via trackMCMatch association map
    "pu",           # Pileup
    "gv",           # GenVertex (the daughters)
    "gvs",          # GenVertex (the daughters)
    "pv",           # PrimaryVertex
    "sv",           # SecondaryVertex
    "matchsv_gv",   # SV matched to GV
    "matchsv_gvs",  # SV matched to GV w/Sim
    "svt",          # SecondaryVertex w/MTD timing
    "matchsvt_gv",  # SV w/MTD timing matched to GV
    "matchsvt_gvs", # SV w/MTD timing matched to GV w/Sim
]
trk_branches = [
    "tval",
    "terr",
    "tsig",
    "tqual",
    # "x",
    # "y",
    # "z",
    "pt",
    "pt2",
    # "pterr",
    "eta",
    # "etaerr",
    "phi",
    # "phierr",
    "dxy",
    # "dxyerr",
    "dxysig",
    # "dz",
    # "dzerr",
    # "dzsig",
    "d3d",
    # "d3derr",
    "d3dsig",
    # "d0",
    # "d0err",
    # "d0sig",
    # "charge",
    "pdgId",
    # "chi2",
    # "ndof",
    # "chi2dof",
]

vtx_collections = [
    "gv",           # GenVertex
    "matchgv_sv",   # GV matched to SV
    "matchgv_svt",  # GV matched to SV w/MTD timing
    "gvs",          # GenVertex w/Sim
    "matchgvs_sv",  # GV w/Sim matched to SV
    "matchgvs_svt", # GV w/Sim matched to SV w/MTD timing
    "pv",           # PrimaryVertex
    "sv",           # SecondaryVertex
    "sv_matchgv",   # SecondaryVertex
    "sv_matchgvs",  # SecondaryVertex
    "svt",          # SecondaryVertex w/MTD timing
    "svt_matchgv",  # SecondaryVertex w/MTD timing w/GV
    "svt_matchgvs", # SecondaryVertex w/MTD timing w/GV w/SIM
]
vtx_branches = [
    "timeavg",
    "timerange",
    # "x",
    # "y",
    # "z",
    "xerr",
    "yerr",
    "zerr",
    "xres",
    "yres",
    "zres",
    "pt",
    "pt2",
    "eta",
    "phi",
    "dxy",
    # "dxyerr",
    "dxysig",
    # "dz",
    # "dzerr",
    # "dzsig",
    "d3d",
    # "d3derr",
    "d3dsig",
    "chi2",
    "ndof",
    "chi2dof",
    "ntracks", # (daughters for GV)
    "motherPdgId",
    # "matchdR",
    # "nmatch",
]

# jet_collections = [
#     "all",
#     "match_gen",
#     # "match_gv",
#     "match_gensv",
#     "match_gensvgv",
#     "match_gensvt",
#     "match_gensvtgv",
# ]
# jet_branches = [
#     "pt",
#     "eta",
#     "phi",
#     "hadflav",
#     # "partflav",
#     "nmatch",
# ]

evt_branches = [
    "nGV",
    "nGVs",
    "nPV",
    "nSV",
    "nSVt",
    "nClusters",
    "nClusterst",
]

def getBranchNames():
    n = []
    # for collection in gp_collections:
    #     for branch in gp_branches:
    #         name = "gp_" + collection + "_" + branch
    #         n.append(name)
    for collection in trk_collections:
        for branch in trk_branches:
            name = "trk_" + collection + "_" + branch
            n.append(name)
    for collection in vtx_collections:
        for branch in vtx_branches:
            name = "vtx_" + collection + "_" + branch
            n.append(name)
    # for collection in jet_collections:
    #     for branch in jet_branches:
    #         name = "jet_" + collection + "_" + branch
    #         n.append(name)
    for branch in evt_branches:
        name = "evt_" + branch
        n.append(name)
    return n

branch_names = getBranchNames()
print("Requested %d branches." % len(branch_names))

Requested 449 branches.


In [11]:
r = {}
b = {}
nbins = 80
r["tval"] = (-0.8, 0.8)
b["tval"] = nbins
r["terr"] = (0, 0.1)
b["terr"] = nbins
r["tsig"] = (-50, 50)
b["tsig"] = nbins
r["tqual"] = (0.0, 1.0)
b["tqual"] = nbins
r["timeavg"] = (-0.8, 0.8)
b["timeavg"] = nbins
r["timerange"] = (0, 0.8)
b["timerange"] = nbins
r["pt"] = (0, 100)
b["pt"] = nbins
r["pt2"] = (0, 100)
b["pt2"] = nbins
r["eta"] = (-4.1, 4.1)
b["eta"] = nbins
r["xerr"] = (-0.1, 0.1)
b["xerr"] = nbins
r["yerr"] = (-0.1, 0.1)
b["yerr"] = nbins
r["zerr"] = (-0.1, 0.1)
b["zerr"] = nbins
r["xres"] = (-0.1, 0.1)
b["xres"] = nbins
r["yres"] = (-0.1, 0.1)
b["yres"] = nbins
r["zres"] = (-0.1, 0.1)
b["zres"] = nbins
r["dxy"] = (0, 1.0)
b["dxy"] = nbins
r["dxysig"] = (0, 3000)
b["dxysig"] = 50
r["d3d"] = (0, 10)
b["d3d"] = nbins
r["d3dsig"] = (0, 1000)
b["d3dsig"] = nbins
r["ntracks"] = (0, 30)
b["ntracks"] = 30
r["motherPdgId"] = (0, 5)
b["motherPdgId"] = 5
r["motherPdgId"] = (0, 5)
b["motherPdgId"] = 5
r["nGV"] = (0, 10)
b["nGV"] = 10
r["nGVs"] = (0, 10)
b["nGVs"] = 10
r["nPV"] = (0, 200)
b["nPV"] = 200
r["nSV"] = (0, 200)
b["nSV"] = 200
r["nSVt"] = (0, 200)
b["nSVt"] = 200
r["nClusters"] = (0, 200)
b["nClusters"] = 200
r["nClusterst"] = (0, 200)
b["nClusterst"] = 200

v2pre = [
    "trk_sv",
    "trk_matchsv_gvs",
    "vtx_sv",
    "vtx_svt",
    "vtx_svt_matchgvs",
]
v2var = [
    "pt_tval",
    "pt2_tval",
    "dxy_tval",
    "dxysig_tval",
    "d3d_tval",
    "d3dsig_tval",
    "pt_tsig",
    "pt2_tsig",
    "dxy_tsig",
    "dxysig_tsig",
    "d3d_tsig",
    "d3dsig_tsig",
    "pt_timeavg",
    "pt2_timeavg",
    "dxy_timeavg",
    "dxysig_timeavg",
    "d3d_timeavg",
    "d3dsig_timeavg",
    "pt_timerange",
    "pt2_timerange",
    "dxy_timerange",
    "dxysig_timerange",
    "d3d_timerange",
    "d3dsig_timerange",
]

# 1d histograms
n = {}
nb = {}
p = {}
pb = {}
for branch in branch_names:
    var = branch.split("_")[-1]
    if var not in b.keys():
        # print("%s not in" % var)
        continue
    # print("%s in" % branch)
    rge = r[var]
    if "vtx_pt" in branch: rge = (0.0, 300)
    n[branch], nb[branch] = np.histogram([], b[var], rge)
    p[branch], pb[branch] = np.histogram([], b[var], rge)

# 2d histograms
n2 = {}
nbx = {}
nby = {}
p2 = {}
pbx = {}
pby = {}
for pre in v2pre:
    for vars in v2var:
        # print(pre + "_" + vars)
        var = vars.split("_")
        n2[pre + "_" + vars], nbx[pre + "_" + vars], nby[pre + "_" + vars] = np.histogram2d([], [], bins=[b[var[0]], b[var[1]]], range=[r[var[0]], r[var[1]]])
        p2[pre + "_" + vars], pbx[pre + "_" + vars], pby[pre + "_" + vars] = np.histogram2d([], [], bins=[b[var[0]], b[var[1]]], range=[r[var[0]], r[var[1]]])

In [12]:
n_benchmark = 20
n_stop = -1
step_size = "200MB"
tree_dir = "deepntuplizer/tree"
# parent_dir = "/eos/user/e/etsai/workspace/DeepNTuples_CMSSW_14_1_0_pre0/src/DeepNTuples/DeepNtuplizer/production/"
# sample_files = [
#     parent_dir + "output_TTToHadronic_noPU.root:deepntuplizer/tree",
#     parent_dir + "output_TTToHadronic_PU200.root:deepntuplizer/tree",
# ]
parent_dir = "/eos/cms/store/user/etsai/DeepNTuples/"
sample_files = [
    parent_dir + "TTToHadronic_noPU/240503_093850/0000/output_0_*.root:" + tree_dir,
    parent_dir + "TTToHadronic_PU200/240503_093944/0000/output_0_*.root:" + tree_dir,
]
sample_name = ["TTnoPU", "TTPU200"]

for s, files in enumerate(sample_files):
    start = time.time()
    n_events = 0
    iter = 0
    loop_end = time.time()
    for array in uproot.iterate(files, branch_names, step_size=step_size):
        if n_stop >= 0 and iter >= n_stop: break
        if iter % n_benchmark == 0:
            print("Processing the %sth iteration." % iter)
            loop_start = time.time()
            print("Array(s) read time was %.2f seconds." % (loop_start-loop_end))

        # 1d histograms
        for branch in branch_names:
            tmp_array = np.array(ak.flatten(array[branch], axis=None))
            var = branch.split("_")[-1]
            if len(tmp_array) == 0 or var not in b.keys(): continue # b.keys() and r.keys() are the same
            if var == "motherPdgId": tmp_array = convertToHadronType(tmp_array)
            rge = r[var]
            if "vtx_pt" in branch: rge = (0.0, 300)
            tmp_hist = np.histogram(tmp_array, b[var], rge)
            if sample_name[s] == "TTnoPU": n[branch] = n[branch] + tmp_hist[0]
            else:                          p[branch] = p[branch] + tmp_hist[0]

        # 2d histograms
        for pre in v2pre:
            for vars in v2var:
                var = vars.split("_")
                branchx = pre + "_" + var[0]
                branchy = pre + "_" + var[1]
                if var[0] == "pt2" and branchy in branch_names:
                    tmp_xarr = np.array(ak.flatten(array[pre + "_pt"], axis=None))
                    tmp_xarr = [x*x for x in tmp_xarr]
                elif branchx not in branch_names or branchy not in branch_names:
                    # print(pre + "_" + vars)
                    continue
                else:
                    tmp_xarr = np.array(ak.flatten(array[branchx], axis=None))
                tmp_yarr = np.array(ak.flatten(array[branchy], axis=None))
                tmp_h2d = np.histogram2d(tmp_xarr, tmp_yarr, bins=[b[var[0]], b[var[1]]], range=[r[var[0]], r[var[1]]])
                if sample_name[s] == "TTnoPU": n2[pre + "_" + vars] = n2[pre + "_" + vars] + tmp_h2d[0]
                else:                          p2[pre + "_" + vars] = p2[pre + "_" + vars] + tmp_h2d[0]

        n_events += len(array["evt_nClusters"]) # pick an arbitrary event-level variable

        loop_end = time.time()
        if iter % n_benchmark == 0:
            print("Loop time was %.2f seconds." % (loop_end-loop_start))
        iter += 1
    end = time.time()

    print("File read time was %.2f minutes with %d iterations and a step size of %s." % ((end-start)/60, iter, step_size))
    print("There are %d %s events.\n" % (n_events, sample_name[s]))

Processing the 0th iteration.
Array(s) read time was 4.49 seconds.
Loop time was 1.43 seconds.
Processing the 20th iteration.
Array(s) read time was 5.27 seconds.
Loop time was 0.89 seconds.
Processing the 40th iteration.
Array(s) read time was 6.82 seconds.
Loop time was 1.39 seconds.
Processing the 60th iteration.
Array(s) read time was 4.79 seconds.
Loop time was 0.91 seconds.
Processing the 80th iteration.
Array(s) read time was 5.10 seconds.
Loop time was 0.91 seconds.
Processing the 100th iteration.
Array(s) read time was 5.24 seconds.
Loop time was 0.90 seconds.
Processing the 120th iteration.
Array(s) read time was 6.50 seconds.
Loop time was 1.39 seconds.
Processing the 140th iteration.
Array(s) read time was 4.84 seconds.
Loop time was 0.90 seconds.
Processing the 160th iteration.
Array(s) read time was 5.31 seconds.
Loop time was 0.93 seconds.
Processing the 180th iteration.
Array(s) read time was 6.32 seconds.
Loop time was 1.40 seconds.
Processing the 200th iteration.
Arra

In [13]:
print("Available histograms:")
# Could also use p and p2
print("1d -----------------")
for key in n.keys():
    if len(n[key]) > 0:
        print(key)
print("2d -----------------")
for key in n2.keys():
    if len(n2[key]) > 0:
        print(key)

Available histograms:
1d -----------------
trk_st_tval
trk_st_terr
trk_st_tsig
trk_st_tqual
trk_st_pt
trk_st_pt2
trk_st_eta
trk_st_dxy
trk_st_dxysig
trk_st_d3d
trk_st_d3dsig
trk_all_tval
trk_all_terr
trk_all_tsig
trk_all_tqual
trk_all_pt
trk_all_pt2
trk_all_eta
trk_all_dxy
trk_all_dxysig
trk_all_d3d
trk_all_d3dsig
trk_match_gp_tval
trk_match_gp_terr
trk_match_gp_tsig
trk_match_gp_tqual
trk_match_gp_pt
trk_match_gp_pt2
trk_match_gp_eta
trk_match_gp_dxy
trk_match_gp_dxysig
trk_match_gp_d3d
trk_match_gp_d3dsig
trk_pu_tval
trk_pu_terr
trk_pu_tsig
trk_pu_tqual
trk_pu_pt
trk_pu_pt2
trk_pu_eta
trk_pu_dxy
trk_pu_dxysig
trk_pu_d3d
trk_pu_d3dsig
trk_gv_tval
trk_gv_terr
trk_gv_tsig
trk_gv_tqual
trk_gv_pt
trk_gv_pt2
trk_gv_eta
trk_gv_dxy
trk_gv_dxysig
trk_gv_d3d
trk_gv_d3dsig
trk_gvs_tval
trk_gvs_terr
trk_gvs_tsig
trk_gvs_tqual
trk_gvs_pt
trk_gvs_pt2
trk_gvs_eta
trk_gvs_dxy
trk_gvs_dxysig
trk_gvs_d3d
trk_gvs_d3dsig
trk_pv_tval
trk_pv_terr
trk_pv_tsig
trk_pv_tqual
trk_pv_pt
trk_pv_pt2
trk_pv_eta
tr

In [73]:
print("             TTnoPU,   TTPU200  \n")

print("There are %d, %d SimTracks."                                                                   % (np.sum(n["trk_st_pt"]), np.sum(p["trk_st_pt"])))
print("There are  %d,  %d reco tracks."                                                               % (np.sum(n["trk_all_pt"]), np.sum(p["trk_all_pt"])))
print("There are  %d,   %d reco tracks matched to MC tracks."                                         % (np.sum(n["trk_match_gp_pt"]), np.sum(p["trk_match_gp_pt"])))
print("There are         %d,         %d pileup tracks."                                               % (np.sum(n["trk_pu_pt"]), np.sum(p["trk_pu_pt"])))
print("There are   %d,   %d GV daughters."                                                            % (np.sum(n["trk_gv_pt"]), np.sum(p["trk_gv_pt"])))
print("There are   %d,    %d GV daughters in GVs matched to SimTracks."                               % (np.sum(n["trk_gvs_pt"]), np.sum(p["trk_gvs_pt"])))
print("There are  %d,  %d PV tracks."                                                                 % (np.sum(n["trk_pv_pt"]), np.sum(p["trk_pv_pt"])))
print("There are  %d,  %d tracks in SVs."                                                             % (np.sum(n["trk_sv_pt"]), np.sum(p["trk_sv_pt"])))
print("There are    %d,    %d tracks in SVs matched to GVs."                                          % (np.sum(n["trk_matchsv_gv_pt"]), np.sum(p["trk_matchsv_gv_pt"])))
print("There are    %d,    %d tracks in SVs matched to GVs matched to SimTracks."                     % (np.sum(n["trk_matchsv_gvs_pt"]), np.sum(p["trk_matchsv_gvs_pt"])))
print("There are  %d,   %d tracks in SVs w/time range cut."                                           % (np.sum(n["trk_svt_pt"]), np.sum(p["trk_svt_pt"])))
print("There are    %d,    %d tracks in SVs w/time range cut matched to GVs."                         % (np.sum(n["trk_matchsvt_gv_pt"]), np.sum(p["trk_matchsvt_gv_pt"])))
print("There are    %d,    %d tracks in SVs w/time range cut matched to GVs matched to SimTracks.\n " % (np.sum(n["trk_matchsvt_gvs_pt"]), np.sum(p["trk_matchsvt_gvs_pt"])))

print("There are   %d,    %d GVs."                                                                    % (np.sum(n["vtx_gv_pt"]), np.sum(p["vtx_gv_pt"])))
print("There are    %d,     %d GVs matched to reco SVs."                                              % (np.sum(n["vtx_matchgv_sv_pt"]), np.sum(p["vtx_matchgv_sv_pt"])))
print("There are    %d,     %d GVs matched to reco SVs w/time range cut."                             % (np.sum(n["vtx_matchgv_svt_pt"]), np.sum(p["vtx_matchgv_svt_pt"])))
print("There are    %d,    %d GVs matched to SimTracks."                                              % (np.sum(n["vtx_gvs_pt"]), np.sum(p["vtx_gvs_pt"])))
print("There are    %d,     %d GVs matched to SimTracks and reco SVs."                                % (np.sum(n["vtx_matchgvs_sv_pt"]), np.sum(p["vtx_matchgvs_sv_pt"])))
print("There are    %d,     %d GVs matched to SimTracks and reco SVs w/time range cut."               % (np.sum(n["vtx_matchgvs_svt_pt"]), np.sum(p["vtx_matchgvs_svt_pt"])))
print("There are    %d,   %d PVs."                                                                    % (np.sum(n["vtx_pv_ntracks"]), np.sum(p["vtx_pv_ntracks"])))
print("There are   %d,   %d SVs."                                                                     % (np.sum(n["vtx_sv_pt"]), np.sum(p["vtx_sv_pt"])))
print("There are   %d,   %d SVs w/GV."                                                                     % (np.sum(n["vtx_sv_matchgv_pt"]), np.sum(p["vtx_sv_matchgv_pt"])))
print("There are   %d,   %d SVs w/GVs."                                                                     % (np.sum(n["vtx_sv_matchgvs_pt"]), np.sum(p["vtx_sv_matchgvs_pt"])))
print("There are   %d,   %d SVs w/time range cut."                                                                     % (np.sum(n["vtx_svt_pt"]), np.sum(p["vtx_svt_pt"])))
print("There are   %d,   %d SVs w/time range cut w/GV."                                                    % (np.sum(n["vtx_svt_matchgv_pt"]), np.sum(p["vtx_svt_matchgv_pt"])))
print("There are   %d,   %d SVs w/time range cut w/GVs."                                                    % (np.sum(n["vtx_svt_matchgvs_pt"]), np.sum(p["vtx_svt_matchgvs_pt"])))

             TTnoPU,   TTPU200  

There are 936322552, 298816957 SimTracks.
There are  12354197,  96549495 reco tracks.
There are  10034243,   3290735 reco tracks matched to MC tracks.
There are         0,         0 pileup tracks.
There are   3961886,   1264146 GV daughters.
There are   1904461,    608024 GV daughters in GVs matched to SimTracks.
There are  10247021,  81082035 PV tracks.
There are  15093188,  24905367 tracks in SVs.
There are    401487,    127081 tracks in SVs matched to GVs.
There are    399298,    126391 tracks in SVs matched to GVs matched to SimTracks.
There are  11690256,   8767790 tracks in SVs w/time range cut.
There are    397243,    118509 tracks in SVs w/time range cut matched to GVs.
There are    395187,    117889 tracks in SVs w/time range cut matched to GVs matched to SimTracks.
 
There are   1243586,    396710 GVs.
There are    108755,     33236 GVs matched to reco SVs.
There are    105783,     31216 GVs matched to reco SVs w/time range cut.
There are    

In [15]:
plot_start = time.time()

In [16]:
era = "Phase2"
save_dir = "/eos/home-b/btvweb/www/Offline/Phase2/etsai/" + era + "/"
trk_path = save_dir + "Track/"
gv_path  = save_dir + "GenVertex/"
pv_path  = save_dir + "PrimaryVertex/"
sv_path  = save_dir + "SecondaryVertex/"
eff_path = save_dir + "Efficiency/"
jet_path = save_dir + "Jet/"

xticks = ["B meson", "B baryon", "C meson", "C baryon", "S baryon"]

---
# 1D histograms

## Compare vertex tracks

In [71]:
labels = ["t$\\bar{t}$ no PU GVs", "t$\\bar{t}$ no PU SVt", "t$\\bar{t}$ PU 200 GVs", "t$\\bar{t}$ PU 200 SVt"]

var = "pt"
hists = [n["trk_gvs_"+var], n["trk_svt_"+var], p["trk_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["trk_gvs_"+var], labels, "Track $p_{T}$ [GeV]", "trk_svt_"+var, gv_path, log=True)
var = "pt2"
hists = [n["trk_gvs_"+var], n["trk_svt_"+var], p["trk_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["trk_gvs_"+var], labels, "Track $p_{T}^2$ [GeV]^2", "trk_svt_"+var, gv_path, log=True)
var = "eta"
hists = [n["trk_gvs_"+var], n["trk_svt_"+var], p["trk_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["trk_gvs_"+var], labels, "Track $\eta$", "trk_svt_"+var, gv_path, log=True)

labels = ["t$\\bar{t}$ no PU GV daughters", "t$\\bar{t}$ no PU SimTrack", "t$\\bar{t}$ PU 200 GV daughters", "t$\\bar{t}$ PU 200 SimTrack"]

var = "pt"
hists = [n["trk_gv_"+var], n["trk_st_"+var], p["trk_gv_"+var], p["trk_st_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "Track $p_{T}$ [GeV]", "trk_st_"+var, gv_path, log=True)
var = "pt2"
hists = [n["trk_gv_"+var], n["trk_st_"+var], p["trk_gv_"+var], p["trk_st_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "Track $p_{T}^2$ [GeV]^2", "trk_st_"+var, gv_path, log=True)
var = "eta"
hists = [n["trk_gv_"+var], n["trk_st_"+var], p["trk_gv_"+var], p["trk_st_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "Track $\eta$", "trk_st_"+var, gv_path, log=True)

## Compare vertices

In [18]:
labels = ["t$\\bar{t}$ no PU GVs", "t$\\bar{t}$ no PU SVt", "t$\\bar{t}$ PU 200 GVs", "t$\\bar{t}$ PU 200 SVt"]

var = "pt"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $p_{T}$ [GeV]", "v_"+var, gv_path, log=True)
var = "pt2"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $p_{T}^2$ [GeV]^2", "v_"+var, gv_path, log=True)
var = "eta"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $\eta$", "v_"+var, gv_path, log=True)
var = "dxy"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $d_{xy}$ [cm]", "v_"+var, gv_path, log=True)
var = "dxysig"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $d_{xy}$ significance", "v_"+var, gv_path, log=True)
var = "d3d"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $d_{3d}$ [cm]", "v_"+var, gv_path, log=True)
var = "d3dsig"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex $d_{3d}$ significance", "v_"+var, gv_path, log=True)
var = "ntracks"
hists = [n["vtx_gvs_"+var], n["vtx_svt_"+var], p["vtx_gvs_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "Vertex track multiplicity", "v_"+var, gv_path, log=True)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)


## Gen vertex tracks

In [19]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/SIM match", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/SIM match"]

var = "pt"
hists = [n["trk_gv_"+var], n["trk_gvs_"+var], p["trk_gv_"+var], p["trk_gvs_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "GV daughter $p_{T}$ [GeV]", "trk_"+var, gv_path, log=True)
var = "pt2"
hists = [n["trk_gv_"+var], n["trk_gvs_"+var], p["trk_gv_"+var], p["trk_gvs_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "GV daughter $p_{T}^2$ [GeV]^2", "trk_"+var, gv_path, log=True)
var = "eta"
hists = [n["trk_gv_"+var], n["trk_gvs_"+var], p["trk_gv_"+var], p["trk_gvs_"+var]]
plotVars(hists, nb["trk_gv_"+var], labels, "GV daughter $\eta$", "trk_"+var, gv_path, log=True)

## Secondary vertex tracks

### Compare tracks with no time range cut to tracks with time range cut

In [20]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/time cut", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/time cut"]

var = "pt"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $p_{T}$ [GeV]", "trk_"+var, sv_path, log=True)
var = "pt2"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $p_{T}^2$ [GeV]^2", "trk_"+var, sv_path, log=True)
var = "eta"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $\eta$", "trk_"+var, sv_path, log=True)
var = "tval"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track time [ns]", "trk_"+var, sv_path, log=True)
var = "terr"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track time error [ns]", "trk_"+var, sv_path, log=True)
var = "tsig"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track time significance", "trk_"+var, sv_path, log=True)
var = "tqual"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track time quality", "trk_"+var, sv_path, log=True)
var = "dxy"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $d_{xy} [cm]$", "trk_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $d_{xy} significance$", "trk_"+var, sv_path, log=True)
var = "d3d"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $d_{3d} [cm]$", "trk_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["trk_sv_"+var], n["trk_svt_"+var], p["trk_sv_"+var], p["trk_svt_"+var]]
plotVars(hists, nb["trk_sv_"+var], labels, "SV track $d_{3d} significance$", "trk_"+var, sv_path, log=True)

### Compare tracks with time range cut to tracks in vertices matched to GVs

In [21]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/GV", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/GV"]

var = "pt"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $p_{T}$ [GeV]", "trk_matchgv_"+var, sv_path, log=True)
var = "pt2"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $p_{T}^2$ [GeV]^2", "trk_matchgv_"+var, sv_path, log=True)
var = "eta"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $\eta$", "trk_matchgv_"+var, sv_path, log=True)
var = "tval"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time [ns]", "trk_matchgv_"+var, sv_path, log=True)
var = "terr"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time error [ns]", "trk_matchgv_"+var, sv_path, log=True)
var = "tsig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time significance", "trk_matchgv_"+var, sv_path, log=True)
var = "tqual"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time quality", "trk_matchgv_"+var, sv_path, log=True)
var = "dxy"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{xy} [cm]$", "trk_matchgv_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{xy} significance$", "trk_matchgv_"+var, sv_path, log=True)
var = "d3d"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{3d} [cm]$", "trk_matchgv_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gv_"+var], p["trk_svt_"+var], p["trk_matchsvt_gv_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{3d} significance$", "trk_matchgv_"+var, sv_path, log=True)

### Compare tracks with time range cut to tracks in vertices matched to GVs w/SIM match

In [22]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/GVs", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/GVs"]

var = "pt"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $p_{T}$ [GeV]", "trk_matchgvs_"+var, sv_path, log=True)
var = "pt2"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $p_{T}^2$ [GeV]^2", "trk_matchgvs_"+var, sv_path, log=True)
var = "eta"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $\eta$", "trk_matchgvs_"+var, sv_path, log=True)
var = "tval"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time [ns]", "trk_matchgvs_"+var, sv_path, log=True)
var = "terr"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time error [ns]", "trk_matchgvs_"+var, sv_path, log=True)
var = "tsig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time significance", "trk_matchgvs_"+var, sv_path, log=True)
var = "tqual"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track time quality", "trk_matchgvs_"+var, sv_path, log=True)
var = "dxy"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{xy} [cm]$", "trk_matchgvs_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{xy} significance$", "trk_matchgvs_"+var, sv_path, log=True)
var = "d3d"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{3d} [cm]$", "trk_matchgvs_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["trk_svt_"+var], n["trk_matchsvt_gvs_"+var], p["trk_svt_"+var], p["trk_matchsvt_gvs_"+var]]
plotVars(hists, nb["trk_svt_"+var], labels, "SV track $d_{3d} significance$", "trk_matchgvs_"+var, sv_path, log=True)

## Gen vertex

### Compare GenVertex to GenVertex w/SIM

In [23]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/SIM match", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/SIM match"]

var = "pt"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $p_{T}$ [GeV]", "gv_"+var, gv_path, log=True)
var = "pt2"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $p_{T}^2$ [GeV]^2", "gv_"+var, gv_path, log=True)
var = "eta"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $\eta$", "gv_"+var, gv_path, log=True)
var = "dxy"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $d_{xy} [cm]$", "gv_"+var, gv_path, log=True)
var = "dxysig"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $d_{xy} significance$", "gv_"+var, gv_path, log=True)
var = "d3d"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $d_{3d} [cm]$", "gv_"+var, gv_path, log=True)
var = "d3dsig"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV $d_{3d} significance$", "gv_"+var, gv_path, log=True)
var = "ntracks"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV daughter multiplicity", "gv_"+var, gv_path, log=True)
var = "motherPdgId"
hists = [n["vtx_gv_"+var], n["vtx_gvs_"+var], p["vtx_gv_"+var], p["vtx_gvs_"+var]]
plotVars(hists, nb["vtx_gv_"+var], labels, "GV mother PDG ID", "gv_"+var, gv_path, log=True, xticks=xticks)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  if log: plt.ylim(top=max_height*300)


### Compare GenVertex w/SIM to GenVertex w/SIM and SV match

In [24]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/SV match", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/SV match"]

var = "pt"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $p_{T}$ [GeV]", "gvs_"+var, gv_path, log=True)
var = "pt2"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $p_{T}^2$ [GeV]^2", "gvs_"+var, gv_path, log=True)
var = "eta"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $\eta$", "gvs_"+var, gv_path, log=True)
var = "dxy"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{xy} [cm]$", "gvs_"+var, gv_path, log=True)
var = "dxysig"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{xy} significance$", "gvs_"+var, gv_path, log=True)
var = "d3d"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{3d} [cm]$", "gvs_"+var, gv_path, log=True)
var = "d3dsig"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{3d} significance$", "gvs_"+var, gv_path, log=True)
var = "ntracks"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV daughter multiplicity", "gvs_"+var, gv_path, log=True)
var = "motherPdgId"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_sv_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_sv_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV mother PDG ID", "gvs_"+var, gv_path, log=True, xticks=xticks)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  if log: plt.ylim(top=max_height*300)


### Compare GenVertex w/SIM to GenVertex w/SIM and SV match w/time range cut

In [25]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/SVt match", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/SVt match"]

var = "pt"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $p_{T}$ [GeV]", "gvs_svt_"+var, gv_path, log=True)
var = "pt2"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $p_{T}^2$ [GeV]^2", "gvs_svt_"+var, gv_path, log=True)
var = "eta"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $\eta$", "gvs_svt_"+var, gv_path, log=True)
var = "dxy"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{xy} [cm]$", "gvs_svt_"+var, gv_path, log=True)
var = "dxysig"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{xy} significance$", "gvs_svt_"+var, gv_path, log=True)
var = "d3d"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{3d} [cm]$", "gvs_svt_"+var, gv_path, log=True)
var = "d3dsig"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV $d_{3d} significance$", "gvs_svt_"+var, gv_path, log=True)
var = "ntracks"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV daughter multiplicity", "gvs_svt_"+var, gv_path, log=True)
var = "motherPdgId"
hists = [n["vtx_gvs_"+var], n["vtx_matchgvs_svt_"+var], p["vtx_gvs_"+var], p["vtx_matchgvs_svt_"+var]]
plotVars(hists, nb["vtx_gvs_"+var], labels, "GV mother PDG ID", "gvs_svt_"+var, gv_path, log=True, xticks=xticks)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  if log: plt.ylim(top=max_height*300)


## Secondary vertex

### Compare SecondaryVertex to SecondaryVertex w/time range cut

In [26]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/time range cut", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU w/time range cut"]

var = "pt"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $p_{T}$ [GeV]", "sv_"+var, sv_path, log=True)
var = "pt2"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $p_{T}^2$ [GeV]^2", "sv_"+var, sv_path, log=True)
var = "eta"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $\eta$", "sv_"+var, sv_path, log=True)
var = "xerr"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $\sigma_{x}$ [cm]", "sv_"+var, sv_path, log=True)
var = "yerr"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $\sigma_{y}$ [cm]", "sv_"+var, sv_path, log=True)
var = "zerr"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $\sigma_{z}$ [cm]", "sv_"+var, sv_path, log=True)
var = "xres"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV x resolution [cm]", "sv_"+var, sv_path, log=True)
var = "yres"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV y resolution [cm]", "sv_"+var, sv_path, log=True)
var = "zres"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV z resolution [cm]", "sv_"+var, sv_path, log=True)
var = "dxy"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $d_{xy} [cm]$", "sv_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $d_{xy} significance$", "sv_"+var, sv_path, log=True)
var = "d3d"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $d_{3d} [cm]$", "sv_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV $d_{3d} significance$", "sv_"+var, sv_path, log=True)
var = "ntracks"
hists = [n["vtx_sv_"+var], n["vtx_svt_"+var], p["vtx_sv_"+var], p["vtx_svt_"+var]]
plotVars(hists, nb["vtx_sv_"+var], labels, "SV track multiplicity", "sv_"+var, sv_path, log=True)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/3864296669.py:14: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  if log: plt.ylim(top=max_height*300)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)


### Compare SecondaryVertex w/time range cut to SecondaryVertex w/time range cut and GV match

In [28]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/GV", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU w/GV"]

var = "pt"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $p_{T}$ [GeV]", "sv_"+var, sv_path, log=True)
var = "pt2"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $p_{T}^2$ [GeV]^2", "sv_"+var, sv_path, log=True)
var = "eta"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\eta$", "sv_"+var, sv_path, log=True)
var = "xerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{x}$ [cm]", "sv_"+var, sv_path, log=True)
var = "yerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{y}$ [cm]", "sv_"+var, sv_path, log=True)
var = "zerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{z}$ [cm]", "sv_"+var, sv_path, log=True)
var = "xres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV x resolution [cm]", "sv_"+var, sv_path, log=True)
var = "yres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV y resolution [cm]", "sv_"+var, sv_path, log=True)
var = "zres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV z resolution [cm]", "sv_"+var, sv_path, log=True)
var = "dxy"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{xy} [cm]$", "sv_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{xy} significance$", "sv_"+var, sv_path, log=True)
var = "d3d"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{3d} [cm]$", "sv_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{3d} significance$", "sv_"+var, sv_path, log=True)
var = "ntracks"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgv_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgv_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV track multiplicity", "sv_"+var, sv_path, log=True)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)


### Compare SecondaryVertex w/time range cut to SecondaryVertex w/time range cut and GV w/SIM match

In [29]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/GV", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU w/GV"]

var = "pt"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $p_{T}$ [GeV]", "sv_"+var, sv_path, log=True)
var = "pt2"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $p_{T}^2$ [GeV]^2", "sv_"+var, sv_path, log=True)
var = "eta"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\eta$", "sv_"+var, sv_path, log=True)
var = "xerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{x}$ [cm]", "sv_"+var, sv_path, log=True)
var = "yerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{y}$ [cm]", "sv_"+var, sv_path, log=True)
var = "zerr"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $\sigma_{z}$ [cm]", "sv_"+var, sv_path, log=True)
var = "xres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV x resolution [cm]", "sv_"+var, sv_path, log=True)
var = "yres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV y resolution [cm]", "sv_"+var, sv_path, log=True)
var = "zres"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV z resolution [cm]", "sv_"+var, sv_path, log=True)
var = "dxy"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{xy} [cm]$", "sv_"+var, sv_path, log=True)
var = "dxysig"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{xy} significance$", "sv_"+var, sv_path, log=True)
var = "d3d"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{3d} [cm]$", "sv_"+var, sv_path, log=True)
var = "d3dsig"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV $d_{3d} significance$", "sv_"+var, sv_path, log=True)
var = "ntracks"
hists = [n["vtx_svt_"+var], n["vtx_svt_matchgvs_"+var], p["vtx_svt_"+var], p["vtx_svt_matchgvs_"+var]]
plotVars(hists, nb["vtx_svt_"+var], labels, "SV track multiplicity", "sv_"+var, sv_path, log=True)

/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)
/tmp/ipykernel_647/1895280519.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return sum / np.sum(hist)


## Vertex multiplicity

In [30]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/SIM match", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU 200 w/SIM match"]
hists = [n["evt_nGV"], n["evt_nGVs"], p["evt_nGV"], p["evt_nGVs"]]
plotVars(hists, nb["evt_nGV"], labels, "GV multiplicity", "nGV", gv_path, log=True)

labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/time range cut", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU w/time range cut"]
hists = [n["evt_nClusters"], n["evt_nClusterst"], p["evt_nClusters"], p["evt_nClusterst"]]
plotVars(hists, nb["evt_nClusters"], labels, "IVF cluster multiplicity", "nclusters", sv_path, log=True)
hists = [n["evt_nSV"], n["evt_nSVt"], p["evt_nSV"], p["evt_nSVt"]]
plotVars(hists, nb["evt_nSV"], labels, "SV multiplicity", "nSV", sv_path, log=True)

hists = [n["evt_nPV"], p["evt_nPV"]]
plotVars(hists, nb["evt_nPV"], ["t$\\bar{t}$ no PU", "t$\\bar{t}$ PU 200"], "PV multiplicity", "nPV", pv_path, log=True)

---
# 2D histograms

## Tracks

In [31]:
plot2D(n2["trk_sv_pt_tval"],     nbx["trk_sv_pt_tval"],     nby["trk_sv_pt_tval"],     "$p_{T}$ [GeV]",         "t [ns]", "noPU_sv_pt_tval",     trk_path)
plot2D(n2["trk_sv_pt2_tval"],    nbx["trk_sv_pt2_tval"],    nby["trk_sv_pt2_tval"],    "$p_{T}^2$ [GeV]^2",     "t [ns]", "noPU_sv_pt2_tval",    trk_path)
plot2D(n2["trk_sv_dxy_tval"],    nbx["trk_sv_dxy_tval"],    nby["trk_sv_dxy_tval"],    "$d_{xy}$ [cm]",         "t [ns]", "noPU_sv_dxy_tval",    trk_path)
plot2D(n2["trk_sv_dxysig_tval"], nbx["trk_sv_dxysig_tval"], nby["trk_sv_dxysig_tval"], "$d_{xy}$ significance", "t [ns]", "noPU_sv_dxysig_tval", trk_path)
plot2D(n2["trk_sv_d3d_tval"],    nbx["trk_sv_d3d_tval"],    nby["trk_sv_d3d_tval"],    "$d_{3D}$ [cm]",         "t [ns]", "noPU_sv_d3d_tval",    trk_path)
plot2D(n2["trk_sv_d3dsig_tval"], nbx["trk_sv_d3dsig_tval"], nby["trk_sv_d3dsig_tval"], "$d_{3D}$ significance", "t [ns]", "noPU_sv_d3dsig_tval", trk_path)

In [32]:
plot2D(p2["trk_sv_pt_tval"],     pbx["trk_sv_pt_tval"],     pby["trk_sv_pt_tval"],     "$p_{T}$ [GeV]",         "t [ns]", "PU200_sv_pt_tval",     trk_path)
plot2D(p2["trk_sv_pt2_tval"],    pbx["trk_sv_pt2_tval"],    pby["trk_sv_pt2_tval"],    "$p_{T}^2$ [GeV]^2",     "t [ns]", "PU200_sv_pt2_tval",    trk_path)
plot2D(p2["trk_sv_dxy_tval"],    pbx["trk_sv_dxy_tval"],    pby["trk_sv_dxy_tval"],    "$d_{xy}$ [cm]",         "t [ns]", "PU200_sv_dxy_tval",    trk_path)
plot2D(p2["trk_sv_dxysig_tval"], pbx["trk_sv_dxysig_tval"], pby["trk_sv_dxysig_tval"], "$d_{xy}$ significance", "t [ns]", "PU200_sv_dxysig_tval", trk_path)
plot2D(p2["trk_sv_d3d_tval"],    pbx["trk_sv_d3d_tval"],    pby["trk_sv_d3d_tval"],    "$d_{3D}$ [cm]",         "t [ns]", "PU200_sv_d3d_tval",    trk_path)
plot2D(p2["trk_sv_d3dsig_tval"], pbx["trk_sv_d3dsig_tval"], pby["trk_sv_d3dsig_tval"], "$d_{3D}$ significance", "t [ns]", "PU200_sv_d3dsig_tval", trk_path)

In [33]:
plot2D(n2["trk_sv_pt_tsig"],     nbx["trk_sv_pt_tsig"],     nby["trk_sv_pt_tsig"],     "$p_{T}$ [GeV]",         "time significance", "noPU_sv_pt_tsig",     trk_path)
plot2D(n2["trk_sv_pt2_tsig"],    nbx["trk_sv_pt2_tsig"],    nby["trk_sv_pt2_tsig"],    "$p_{T}^2$ [GeV]^2",     "time significance", "noPU_sv_pt2_tsig",    trk_path)
plot2D(n2["trk_sv_dxy_tsig"],    nbx["trk_sv_dxy_tsig"],    nby["trk_sv_dxy_tsig"],    "$d_{xy}$ [cm]",         "time significance", "noPU_sv_dxy_tsig",    trk_path)
plot2D(n2["trk_sv_dxysig_tsig"], nbx["trk_sv_dxysig_tsig"], nby["trk_sv_dxysig_tsig"], "$d_{xy}$ significance", "time significance", "noPU_sv_dxysig_tsig", trk_path)
plot2D(n2["trk_sv_d3d_tsig"],    nbx["trk_sv_d3d_tsig"],    nby["trk_sv_d3d_tsig"],    "$d_{3D}$ [cm]",         "time significance", "noPU_sv_d3d_tsig",    trk_path)
plot2D(n2["trk_sv_d3dsig_tsig"], nbx["trk_sv_d3dsig_tsig"], nby["trk_sv_d3dsig_tsig"], "$d_{3D}$ significance", "time significance", "noPU_sv_d3dsig_tsig", trk_path)

In [34]:
plot2D(p2["trk_sv_pt_tsig"],     pbx["trk_sv_pt_tsig"],     pby["trk_sv_pt_tsig"],     "$p_{T}$ [GeV]",         "time significance", "PU200_sv_pt_tsig",     trk_path)
plot2D(p2["trk_sv_pt2_tsig"],    pbx["trk_sv_pt2_tsig"],    pby["trk_sv_pt2_tsig"],    "$p_{T}^2$ [GeV]^2",     "time significance", "PU200_sv_pt2_tsig",    trk_path)
plot2D(p2["trk_sv_dxy_tsig"],    pbx["trk_sv_dxy_tsig"],    pby["trk_sv_dxy_tsig"],    "$d_{xy}$ [cm]",         "time significance", "PU200_sv_dxy_tsig",    trk_path)
plot2D(p2["trk_sv_dxysig_tsig"], pbx["trk_sv_dxysig_tsig"], pby["trk_sv_dxysig_tsig"], "$d_{xy}$ significance", "time significance", "PU200_sv_dxysig_tsig", trk_path)
plot2D(p2["trk_sv_d3d_tsig"],    pbx["trk_sv_d3d_tsig"],    pby["trk_sv_d3d_tsig"],    "$d_{3D}$ [cm]",         "time significance", "PU200_sv_d3d_tsig",    trk_path)
plot2D(p2["trk_sv_d3dsig_tsig"], pbx["trk_sv_d3dsig_tsig"], pby["trk_sv_d3dsig_tsig"], "$d_{3D}$ significance", "time significance", "PU200_sv_d3dsig_tsig", trk_path)

---

In [35]:
plot2D(n2["trk_matchsv_gvs_pt_tval"],     nbx["trk_matchsv_gvs_pt_tval"],     nby["trk_matchsv_gvs_pt_tval"],     "$p_{T}$ [GeV]",         "t [ns]", "noPU_matchsv_gvs_pt_tval",     trk_path)
plot2D(n2["trk_matchsv_gvs_pt2_tval"],    nbx["trk_matchsv_gvs_pt2_tval"],    nby["trk_matchsv_gvs_pt2_tval"],    "$p_{T}^2$ [GeV]^2",     "t [ns]", "noPU_matchsv_gvs_pt2_tval",    trk_path)
plot2D(n2["trk_matchsv_gvs_dxy_tval"],    nbx["trk_matchsv_gvs_dxy_tval"],    nby["trk_matchsv_gvs_dxy_tval"],    "$d_{xy}$ [cm]",         "t [ns]", "noPU_matchsv_gvs_dxy_tval",    trk_path)
plot2D(n2["trk_matchsv_gvs_dxysig_tval"], nbx["trk_matchsv_gvs_dxysig_tval"], nby["trk_matchsv_gvs_dxysig_tval"], "$d_{xy}$ significance", "t [ns]", "noPU_matchsv_gvs_dxysig_tval", trk_path)
plot2D(n2["trk_matchsv_gvs_d3d_tval"],    nbx["trk_matchsv_gvs_d3d_tval"],    nby["trk_matchsv_gvs_d3d_tval"],    "$d_{3D}$ [cm]",         "t [ns]", "noPU_matchsv_gvs_d3d_tval",    trk_path)
plot2D(n2["trk_matchsv_gvs_d3dsig_tval"], nbx["trk_matchsv_gvs_d3dsig_tval"], nby["trk_matchsv_gvs_d3dsig_tval"], "$d_{3D}$ significance", "t [ns]", "noPU_matchsv_gvs_d3dsig_tval", trk_path)

In [36]:
plot2D(p2["trk_matchsv_gvs_pt_tval"],     pbx["trk_matchsv_gvs_pt_tval"],     pby["trk_matchsv_gvs_pt_tval"],     "$p_{T}$ [GeV]",         "t [ns]", "PU200_matchsv_gvs_pt_tval",     trk_path)
plot2D(p2["trk_matchsv_gvs_pt2_tval"],    pbx["trk_matchsv_gvs_pt2_tval"],    pby["trk_matchsv_gvs_pt2_tval"],    "$p_{T}^2$ [GeV]^2",     "t [ns]", "PU200_matchsv_gvs_pt2_tval",    trk_path)
plot2D(p2["trk_matchsv_gvs_dxy_tval"],    pbx["trk_matchsv_gvs_dxy_tval"],    pby["trk_matchsv_gvs_dxy_tval"],    "$d_{xy}$ [cm]",         "t [ns]", "PU200_matchsv_gvs_dxy_tval",    trk_path)
plot2D(p2["trk_matchsv_gvs_dxysig_tval"], pbx["trk_matchsv_gvs_dxysig_tval"], pby["trk_matchsv_gvs_dxysig_tval"], "$d_{xy}$ significance", "t [ns]", "PU200_matchsv_gvs_dxysig_tval", trk_path)
plot2D(p2["trk_matchsv_gvs_d3d_tval"],    pbx["trk_matchsv_gvs_d3d_tval"],    pby["trk_matchsv_gvs_d3d_tval"],    "$d_{3D}$ [cm]",         "t [ns]", "PU200_matchsv_gvs_d3d_tval",    trk_path)
plot2D(p2["trk_matchsv_gvs_d3dsig_tval"], pbx["trk_matchsv_gvs_d3dsig_tval"], pby["trk_matchsv_gvs_d3dsig_tval"], "$d_{3D}$ significance", "t [ns]", "PU200_matchsv_gvs_d3dsig_tval", trk_path)

In [37]:
plot2D(n2["trk_matchsv_gvs_pt_tsig"],     nbx["trk_matchsv_gvs_pt_tsig"],     nby["trk_matchsv_gvs_pt_tsig"],     "$p_{T}$ [GeV]",         "time significance", "noPU_matchsv_gvs_pt_tsig",     trk_path)
plot2D(n2["trk_matchsv_gvs_pt2_tsig"],    nbx["trk_matchsv_gvs_pt2_tsig"],    nby["trk_matchsv_gvs_pt2_tsig"],    "$p_{T}^2$ [GeV]^2",     "time significance", "noPU_matchsv_gvs_pt2_tsig",    trk_path)
plot2D(n2["trk_matchsv_gvs_dxy_tsig"],    nbx["trk_matchsv_gvs_dxy_tsig"],    nby["trk_matchsv_gvs_dxy_tsig"],    "$d_{xy}$ [cm]",         "time significance", "noPU_matchsv_gvs_dxy_tsig",    trk_path)
plot2D(n2["trk_matchsv_gvs_dxysig_tsig"], nbx["trk_matchsv_gvs_dxysig_tsig"], nby["trk_matchsv_gvs_dxysig_tsig"], "$d_{xy}$ significance", "time significance", "noPU_matchsv_gvs_dxysig_tsig", trk_path)
plot2D(n2["trk_matchsv_gvs_d3d_tsig"],    nbx["trk_matchsv_gvs_d3d_tsig"],    nby["trk_matchsv_gvs_d3d_tsig"],    "$d_{3D}$ [cm]",         "time significance", "noPU_matchsv_gvs_d3d_tsig",    trk_path)
plot2D(n2["trk_matchsv_gvs_d3dsig_tsig"], nbx["trk_matchsv_gvs_d3dsig_tsig"], nby["trk_matchsv_gvs_d3dsig_tsig"], "$d_{3D}$ significance", "time significance", "noPU_matchsv_gvs_d3dsig_tsig", trk_path)

In [38]:
plot2D(p2["trk_matchsv_gvs_pt_tsig"],     pbx["trk_matchsv_gvs_pt_tsig"],     pby["trk_matchsv_gvs_pt_tsig"],     "$p_{T}$ [GeV]",         "time significance", "PU200_matchsv_gvs_pt_tsig",     trk_path)
plot2D(p2["trk_matchsv_gvs_pt2_tsig"],    pbx["trk_matchsv_gvs_pt2_tsig"],    pby["trk_matchsv_gvs_pt2_tsig"],    "$p_{T}^2$ [GeV]^2",     "time significance", "PU200_matchsv_gvs_pt2_tsig",    trk_path)
plot2D(p2["trk_matchsv_gvs_dxy_tsig"],    pbx["trk_matchsv_gvs_dxy_tsig"],    pby["trk_matchsv_gvs_dxy_tsig"],    "$d_{xy}$ [cm]",         "time significance", "PU200_matchsv_gvs_dxy_tsig",    trk_path)
plot2D(p2["trk_matchsv_gvs_dxysig_tsig"], pbx["trk_matchsv_gvs_dxysig_tsig"], pby["trk_matchsv_gvs_dxysig_tsig"], "$d_{xy}$ significance", "time significance", "PU200_matchsv_gvs_dxysig_tsig", trk_path)
plot2D(p2["trk_matchsv_gvs_d3d_tsig"],    pbx["trk_matchsv_gvs_d3d_tsig"],    pby["trk_matchsv_gvs_d3d_tsig"],    "$d_{3D}$ [cm]",         "time significance", "PU200_matchsv_gvs_d3d_tsig",    trk_path)
plot2D(p2["trk_matchsv_gvs_d3dsig_tsig"], pbx["trk_matchsv_gvs_d3dsig_tsig"], pby["trk_matchsv_gvs_d3dsig_tsig"], "$d_{3D}$ significance", "time significance", "PU200_matchsv_gvs_d3dsig_tsig", trk_path)

## Secondary vertex

In [39]:
plot2D(n2["vtx_sv_pt_timeavg"],     nbx["vtx_sv_pt_timeavg"],     nby["vtx_sv_pt_timeavg"],     "$p_{T}$ [GeV]",         "<t> [ns]", "noPU_sv_pt_tavg",     sv_path)
plot2D(n2["vtx_sv_pt2_timeavg"],    nbx["vtx_sv_pt2_timeavg"],    nby["vtx_sv_pt2_timeavg"],    "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "noPU_sv_pt2_tavg",    sv_path)
plot2D(n2["vtx_sv_dxy_timeavg"],    nbx["vtx_sv_dxy_timeavg"],    nby["vtx_sv_dxy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "noPU_sv_dxy_tavg",    sv_path)
plot2D(n2["vtx_sv_dxysig_timeavg"], nbx["vtx_sv_dxysig_timeavg"], nby["vtx_sv_dxysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "noPU_sv_dxysig_tavg", sv_path)
plot2D(n2["vtx_sv_d3d_timeavg"],    nbx["vtx_sv_d3d_timeavg"],    nby["vtx_sv_d3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "noPU_sv_d3d_tavg",    sv_path)
plot2D(n2["vtx_sv_d3dsig_timeavg"], nbx["vtx_sv_d3dsig_timeavg"], nby["vtx_sv_d3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "noPU_sv_d3dsig_tavg", sv_path)

In [40]:
plot2D(p2["vtx_sv_pt_timeavg"],     pbx["vtx_sv_pt_timeavg"],     pby["vtx_sv_pt_timeavg"],     "$p_{T}$ [GeV]",         "<t> [ns]", "PU200_sv_pt_tavg",     sv_path)
plot2D(p2["vtx_sv_pt2_timeavg"],    pbx["vtx_sv_pt2_timeavg"],    pby["vtx_sv_pt2_timeavg"],    "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "PU200_sv_pt2_tavg",    sv_path)
plot2D(p2["vtx_sv_dxy_timeavg"],    pbx["vtx_sv_dxy_timeavg"],    pby["vtx_sv_dxy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "PU200_sv_dxy_tavg",    sv_path)
plot2D(p2["vtx_sv_dxysig_timeavg"], pbx["vtx_sv_dxysig_timeavg"], pby["vtx_sv_dxysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "PU200_sv_dxysig_tavg", sv_path)
plot2D(p2["vtx_sv_d3d_timeavg"],    pbx["vtx_sv_d3d_timeavg"],    pby["vtx_sv_d3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "PU200_sv_d3d_tavg",    sv_path)
plot2D(p2["vtx_sv_d3dsig_timeavg"], pbx["vtx_sv_d3dsig_timeavg"], pby["vtx_sv_d3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "PU200_sv_d3dsig_tavg", sv_path)

In [41]:
plot2D(n2["vtx_sv_pt_timerange"],     nbx["vtx_sv_pt_timerange"],     nby["vtx_sv_pt_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "noPU_sv_pt_trange",     sv_path)
plot2D(n2["vtx_sv_pt2_timerange"],    nbx["vtx_sv_pt2_timerange"],    nby["vtx_sv_pt2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "noPU_sv_pt2_trange",    sv_path)
plot2D(n2["vtx_sv_dxy_timerange"],    nbx["vtx_sv_dxy_timerange"],    nby["vtx_sv_dxy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "noPU_sv_dxy_trange",    sv_path)
plot2D(n2["vtx_sv_dxysig_timerange"], nbx["vtx_sv_dxysig_timerange"], nby["vtx_sv_dxysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "noPU_sv_dxysig_trange", sv_path)
plot2D(n2["vtx_sv_d3d_timerange"],    nbx["vtx_sv_d3d_timerange"],    nby["vtx_sv_d3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "noPU_sv_d3d_trange",    sv_path)
plot2D(n2["vtx_sv_d3dsig_timerange"], nbx["vtx_sv_d3dsig_timerange"], nby["vtx_sv_d3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "noPU_sv_d3dsig_trange", sv_path)

In [42]:
plot2D(p2["vtx_sv_pt_timerange"],     pbx["vtx_sv_pt_timerange"],     nby["vtx_sv_pt_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "PU200_sv_pt_trange",     sv_path)
plot2D(p2["vtx_sv_pt2_timerange"],    pbx["vtx_sv_pt2_timerange"],    pby["vtx_sv_pt2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "PU200_sv_pt2_trange",    sv_path)
plot2D(p2["vtx_sv_dxy_timerange"],    pbx["vtx_sv_dxy_timerange"],    pby["vtx_sv_dxy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "PU200_sv_dxy_trange",    sv_path)
plot2D(p2["vtx_sv_dxysig_timerange"], pbx["vtx_sv_dxysig_timerange"], pby["vtx_sv_dxysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "PU200_sv_dxysig_trange", sv_path)
plot2D(p2["vtx_sv_d3d_timerange"],    pbx["vtx_sv_d3d_timerange"],    pby["vtx_sv_d3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "PU200_sv_d3d_trange",    sv_path)
plot2D(p2["vtx_sv_d3dsig_timerange"], pbx["vtx_sv_d3dsig_timerange"], pby["vtx_sv_d3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "PU200_sv_d3dsig_trange", sv_path)

---

In [43]:
plot2D(n2["vtx_svt_pt_timeavg"],     nbx["vtx_svt_pt_timeavg"],     nby["vtx_svt_pt_timeavg"],     "$p_{T}$ [GeV]",         "<t> [ns]", "noPU_svt_pt_tavg",     sv_path)
plot2D(n2["vtx_svt_pt2_timeavg"],    nbx["vtx_svt_pt2_timeavg"],    nby["vtx_svt_pt2_timeavg"],    "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "noPU_svt_pt2_tavg",    sv_path)
plot2D(n2["vtx_svt_dxy_timeavg"],    nbx["vtx_svt_dxy_timeavg"],    nby["vtx_svt_dxy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "noPU_svt_dxy_tavg",    sv_path)
plot2D(n2["vtx_svt_dxysig_timeavg"], nbx["vtx_svt_dxysig_timeavg"], nby["vtx_svt_dxysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "noPU_svt_dxysig_tavg", sv_path)
plot2D(n2["vtx_svt_d3d_timeavg"],    nbx["vtx_svt_d3d_timeavg"],    nby["vtx_svt_d3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "noPU_svt_d3d_tavg",    sv_path)
plot2D(n2["vtx_svt_d3dsig_timeavg"], nbx["vtx_svt_d3dsig_timeavg"], nby["vtx_svt_d3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "noPU_svt_d3dsig_tavg", sv_path)

In [44]:
plot2D(p2["vtx_svt_pt_timeavg"],     pbx["vtx_svt_pt_timeavg"],     pby["vtx_svt_pt_timeavg"],     "$p_{T}$ [GeV]",         "<t> [ns]", "PU200_svt_pt_tavg",     sv_path)
plot2D(p2["vtx_svt_pt2_timeavg"],    pbx["vtx_svt_pt2_timeavg"],    pby["vtx_svt_pt2_timeavg"],    "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "PU200_svt_pt2_tavg",    sv_path)
plot2D(p2["vtx_svt_dxy_timeavg"],    pbx["vtx_svt_dxy_timeavg"],    pby["vtx_svt_dxy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "PU200_svt_dxy_tavg",    sv_path)
plot2D(p2["vtx_svt_dxysig_timeavg"], pbx["vtx_svt_dxysig_timeavg"], pby["vtx_svt_dxysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "PU200_svt_dxysig_tavg", sv_path)
plot2D(p2["vtx_svt_d3d_timeavg"],    pbx["vtx_svt_d3d_timeavg"],    pby["vtx_svt_d3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "PU200_svt_d3d_tavg",    sv_path)
plot2D(p2["vtx_svt_d3dsig_timeavg"], pbx["vtx_svt_d3dsig_timeavg"], pby["vtx_svt_d3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "PU200_svt_d3dsig_tavg", sv_path)

In [45]:
plot2D(n2["vtx_svt_pt_timerange"],     nbx["vtx_svt_pt_timerange"],     nby["vtx_svt_pt_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "noPU_svt_pt_trange",     sv_path)
plot2D(n2["vtx_svt_pt2_timerange"],    nbx["vtx_svt_pt2_timerange"],    nby["vtx_svt_pt2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "noPU_svt_pt2_trange",    sv_path)
plot2D(n2["vtx_svt_dxy_timerange"],    nbx["vtx_svt_dxy_timerange"],    nby["vtx_svt_dxy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "noPU_svt_dxy_trange",    sv_path)
plot2D(n2["vtx_svt_dxysig_timerange"], nbx["vtx_svt_dxysig_timerange"], nby["vtx_svt_dxysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "noPU_svt_dxysig_trange", sv_path)
plot2D(n2["vtx_svt_d3d_timerange"],    nbx["vtx_svt_d3d_timerange"],    nby["vtx_svt_d3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "noPU_svt_d3d_trange",    sv_path)
plot2D(n2["vtx_svt_d3dsig_timerange"], nbx["vtx_svt_d3dsig_timerange"], nby["vtx_svt_d3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "noPU_svt_d3dsig_trange", sv_path)

In [46]:
plot2D(p2["vtx_svt_pt_timerange"],     pbx["vtx_svt_pt_timerange"],     nby["vtx_svt_pt_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "PU200_svt_pt_trange",     sv_path)
plot2D(p2["vtx_svt_pt2_timerange"],    pbx["vtx_svt_pt2_timerange"],    pby["vtx_svt_pt2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "PU200_svt_pt2_trange",    sv_path)
plot2D(p2["vtx_svt_dxy_timerange"],    pbx["vtx_svt_dxy_timerange"],    pby["vtx_svt_dxy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "PU200_svt_dxy_trange",    sv_path)
plot2D(p2["vtx_svt_dxysig_timerange"], pbx["vtx_svt_dxysig_timerange"], pby["vtx_svt_dxysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "PU200_svt_dxysig_trange", sv_path)
plot2D(p2["vtx_svt_d3d_timerange"],    pbx["vtx_svt_d3d_timerange"],    pby["vtx_svt_d3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "PU200_svt_d3d_trange",    sv_path)
plot2D(p2["vtx_svt_d3dsig_timerange"], pbx["vtx_svt_d3dsig_timerange"], pby["vtx_svt_d3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "PU200_svt_d3dsig_trange", sv_path)

---

In [57]:
# plot2D(n2["vtx_svt_matchgvs_pt_timeavg"],    nbx["vtx_svt_matchgvs_pt_timeavg"],    nby["vtx_svt_matchgvs_pt_timeavg"],    "$p_{T}$ [GeV]",         "<t> [ns]", "noPU_svt_matchgvs_pt_tavg",     sv_path)
# plot2D(n2["vtx_svt_matchgvs_pt2_timeavg"],   nbx["vtx_svt_matchgvs_pt2_timeavg"],   nby["vtx_svt_matchgvs_pt2_timeavg"],   "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "noPU_svt_matchgvs_t2_tavg",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_xy_timeavg"],    nbx["vtx_svt_matchgvs_xy_timeavg"],    nby["vtx_svt_matchgvs_xy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "noPU_svt_matchgvs_xy_tavg",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_xysig_timeavg"], nbx["vtx_svt_matchgvs_xysig_timeavg"], nby["vtx_svt_matchgvs_xysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "noPU_svt_matchgvs_xysig_tavg", sv_path)
# plot2D(n2["vtx_svt_matchgvs_3d_timeavg"],    nbx["vtx_svt_matchgvs_3d_timeavg"],    nby["vtx_svt_matchgvs_3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "noPU_svt_matchgvs_3d_tavg",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_3dsig_timeavg"], nbx["vtx_svt_matchgvs_3dsig_timeavg"], nby["vtx_svt_matchgvs_3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "noPU_svt_matchgvs_3dsig_tavg", sv_path)

In [58]:
# plot2D(p2["vtx_svt_matchgvs_t_timeavg"],     pbx["vtx_svt_matchgvs_t_timeavg"],     pby["vtx_svt_matchgvs_t_timeavg"],     "$p_{T}$ [GeV]",         "<t> [ns]", "PU200_svt_matchgvs_t_tavg",     sv_path)
# plot2D(p2["vtx_svt_matchgvs_t2_timeavg"],    pbx["vtx_svt_matchgvs_t2_timeavg"],    pby["vtx_svt_matchgvs_t2_timeavg"],    "$p_{T}^2$ [GeV]^2",     "<t> [ns]", "PU200_svt_matchgvs_t2_tavg",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_xy_timeavg"],    pbx["vtx_svt_matchgvs_xy_timeavg"],    pby["vtx_svt_matchgvs_xy_timeavg"],    "$d_{xy}$ [cm]",         "<t> [ns]", "PU200_svt_matchgvs_xy_tavg",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_xysig_timeavg"], pbx["vtx_svt_matchgvs_xysig_timeavg"], pby["vtx_svt_matchgvs_xysig_timeavg"], "$d_{xy}$ significance", "<t> [ns]", "PU200_svt_matchgvs_xysig_tavg", sv_path)
# plot2D(p2["vtx_svt_matchgvs_3d_timeavg"],    pbx["vtx_svt_matchgvs_3d_timeavg"],    pby["vtx_svt_matchgvs_3d_timeavg"],    "$d_{3D}$ [cm]",         "<t> [ns]", "PU200_svt_matchgvs_3d_tavg",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_3dsig_timeavg"], pbx["vtx_svt_matchgvs_3dsig_timeavg"], pby["vtx_svt_matchgvs_3dsig_timeavg"], "$d_{3D}$ significance", "<t> [ns]", "PU200_svt_matchgvs_3dsig_tavg", sv_path)

In [59]:
# plot2D(n2["vtx_svt_matchgvs_t_timerange"],     nbx["vtx_svt_matchgvs_t_timerange"],     nby["vtx_svt_matchgvs_t_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "noPU_svt_matchgvs_t_trange",     sv_path)
# plot2D(n2["vtx_svt_matchgvs_t2_timerange"],    nbx["vtx_svt_matchgvs_t2_timerange"],    nby["vtx_svt_matchgvs_t2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "noPU_svt_matchgvs_t2_trange",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_xy_timerange"],    nbx["vtx_svt_matchgvs_xy_timerange"],    nby["vtx_svt_matchgvs_xy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "noPU_svt_matchgvs_xy_trange",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_xysig_timerange"], nbx["vtx_svt_matchgvs_xysig_timerange"], nby["vtx_svt_matchgvs_xysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "noPU_svt_matchgvs_xysig_trange", sv_path)
# plot2D(n2["vtx_svt_matchgvs_3d_timerange"],    nbx["vtx_svt_matchgvs_3d_timerange"],    nby["vtx_svt_matchgvs_3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "noPU_svt_matchgvs_3d_trange",    sv_path)
# plot2D(n2["vtx_svt_matchgvs_3dsig_timerange"], nbx["vtx_svt_matchgvs_3dsig_timerange"], nby["vtx_svt_matchgvs_3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "noPU_svt_matchgvs_3dsig_trange", sv_path)

In [60]:
# plot2D(p2["vtx_svt_matchgvs_t_timerange"],     pbx["vtx_svt_matchgvs_t_timerange"],     nby["vtx_svt_matchgvs_t_timerange"],     "$p_{T}$ [GeV]",         "$\Delta$t [ns]", "PU200_svt_matchgvs_t_trange",     sv_path)
# plot2D(p2["vtx_svt_matchgvs_t2_timerange"],    pbx["vtx_svt_matchgvs_t2_timerange"],    pby["vtx_svt_matchgvs_t2_timerange"],    "$p_{T}^2$ [GeV]^2",     "$\Delta$t [ns]", "PU200_svt_matchgvs_t2_trange",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_xy_timerange"],    pbx["vtx_svt_matchgvs_xy_timerange"],    pby["vtx_svt_matchgvs_xy_timerange"],    "$d_{xy}$ [cm]",         "$\Delta$t [ns]", "PU200_svt_matchgvs_xy_trange",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_xysig_timerange"], pbx["vtx_svt_matchgvs_xysig_timerange"], pby["vtx_svt_matchgvs_xysig_timerange"], "$d_{xy}$ significance", "$\Delta$t [ns]", "PU200_svt_matchgvs_xysig_trange", sv_path)
# plot2D(p2["vtx_svt_matchgvs_3d_timerange"],    pbx["vtx_svt_matchgvs_3d_timerange"],    pby["vtx_svt_matchgvs_3d_timerange"],    "$d_{3D}$ [cm]",         "$\Delta$t [ns]", "PU200_svt_matchgvs_3d_trange",    sv_path)
# plot2D(p2["vtx_svt_matchgvs_3dsig_timerange"], pbx["vtx_svt_matchgvs_3dsig_timerange"], pby["vtx_svt_matchgvs_3dsig_timerange"], "$d_{3D}$ significance", "$\Delta$t [ns]", "PU200_svt_matchgvs_3dsig_trange", sv_path)

---
# Efficiencies

In [61]:
labels = ["t$\\bar{t}$ no PU", "t$\\bar{t}$ no PU w/time range cut", "t$\\bar{t}$ PU 200", "t$\\bar{t}$ PU w/time range cut"]

In [62]:
numerators   = [n["vtx_matchgv_sv_pt"], n["vtx_matchgv_svt_pt"], p["vtx_matchgv_sv_pt"], p["vtx_matchgv_svt_pt"]]
denominators = [n["vtx_gv_pt"],         n["vtx_gv_pt"],          p["vtx_gv_pt"],         p["vtx_gv_pt"]]
plotEffs(numerators, denominators, nb["vtx_gv_pt"], labels, "$p_{T}$ [GeV]", "vtx_matchgv_pt", eff_path)
numerators   = [n["vtx_matchgvs_sv_pt"], n["vtx_matchgvs_svt_pt"], p["vtx_matchgvs_sv_pt"], p["vtx_matchgvs_svt_pt"]]
denominators = [n["vtx_gvs_pt"],         n["vtx_gvs_pt"],          p["vtx_gvs_pt"],         p["vtx_gvs_pt"]]
plotEffs(numerators, denominators, nb["vtx_gvs_pt"], labels, "$p_{T}$ [GeV]", "vtx_matchgvs_pt", eff_path)

/tmp/ipykernel_647/3366293035.py:7: RuntimeWarning: invalid value encountered in divide
  eff = numerators[i] / denominators[i]
/tmp/ipykernel_647/3366293035.py:7: RuntimeWarning: invalid value encountered in divide
  eff = numerators[i] / denominators[i]


In [63]:
numerators   = [n["vtx_matchgv_sv_eta"], n["vtx_matchgv_svt_eta"], p["vtx_matchgv_sv_eta"], p["vtx_matchgv_svt_eta"]]
denominators = [n["vtx_gv_eta"],         n["vtx_gv_eta"],          p["vtx_gv_eta"],         p["vtx_gv_eta"]]
plotEffs(numerators, denominators, nb["vtx_gv_eta"], labels, "$\eta$", "vtx_matchgv_eta", eff_path)
numerators   = [n["vtx_matchgvs_sv_eta"], n["vtx_matchgvs_svt_eta"], p["vtx_matchgvs_sv_eta"], p["vtx_matchgvs_svt_eta"]]
denominators = [n["vtx_gvs_eta"],         n["vtx_gvs_eta"],          p["vtx_gvs_eta"],         p["vtx_gvs_eta"]]
plotEffs(numerators, denominators, nb["vtx_gvs_eta"], labels, "$\eta$", "vtx_matchgvs_eta", eff_path)

In [64]:
numerators   = [n["vtx_matchgv_sv_dxy"], n["vtx_matchgv_svt_dxy"], p["vtx_matchgv_sv_dxy"], p["vtx_matchgv_svt_dxy"]]
denominators = [n["vtx_gv_dxy"],         n["vtx_gv_dxy"],          p["vtx_gv_dxy"],         p["vtx_gv_dxy"]]
plotEffs(numerators, denominators, nb["vtx_gv_dxy"], labels, "$d_{xy}$ [cm]", "vtx_matchgv_dxy", eff_path)
numerators   = [n["vtx_matchgvs_sv_dxy"], n["vtx_matchgvs_svt_dxy"], p["vtx_matchgvs_sv_dxy"], p["vtx_matchgvs_svt_dxy"]]
denominators = [n["vtx_gvs_dxy"],         n["vtx_gvs_dxy"],          p["vtx_gvs_dxy"],         p["vtx_gvs_dxy"]]
plotEffs(numerators, denominators, nb["vtx_gvs_dxy"], labels, "$d_{xy}$ [cm]", "vtx_matchgvs_dxy", eff_path)

In [65]:
numerators   = [n["vtx_matchgv_sv_dxysig"], n["vtx_matchgv_svt_dxysig"], p["vtx_matchgv_sv_dxysig"], p["vtx_matchgv_svt_dxysig"]]
denominators = [n["vtx_gv_dxysig"],         n["vtx_gv_dxysig"],          p["vtx_gv_dxysig"],         p["vtx_gv_dxysig"]]
plotEffs(numerators, denominators, nb["vtx_gv_dxysig"], labels, "$d_{xy}$ significance", "vtx_matchgv_dxysig", eff_path)
numerators   = [n["vtx_matchgvs_sv_dxysig"], n["vtx_matchgvs_svt_dxysig"], p["vtx_matchgvs_sv_dxysig"], p["vtx_matchgvs_svt_dxysig"]]
denominators = [n["vtx_gvs_dxysig"],         n["vtx_gvs_dxysig"],          p["vtx_gvs_dxysig"],         p["vtx_gvs_dxysig"]]
plotEffs(numerators, denominators, nb["vtx_gvs_dxysig"], labels, "$d_{xy}$ significance", "vtx_matchgvs_dxysig", eff_path)

In [66]:
numerators   = [n["vtx_matchgv_sv_d3d"], n["vtx_matchgv_svt_d3d"], p["vtx_matchgv_sv_d3d"], p["vtx_matchgv_svt_d3d"]]
denominators = [n["vtx_gv_d3d"],         n["vtx_gv_d3d"],          p["vtx_gv_d3d"],         p["vtx_gv_d3d"]]
plotEffs(numerators, denominators, nb["vtx_gv_d3d"], labels, "$d_{3D}$ [cm]", "vtx_matchgv_d3d", eff_path)
numerators   = [n["vtx_matchgvs_sv_d3d"], n["vtx_matchgvs_svt_d3d"], p["vtx_matchgvs_sv_d3d"], p["vtx_matchgvs_svt_d3d"]]
denominators = [n["vtx_gvs_d3d"],         n["vtx_gvs_d3d"],          p["vtx_gvs_d3d"],         p["vtx_gvs_d3d"]]
plotEffs(numerators, denominators, nb["vtx_gvs_d3d"], labels, "$d_{3D}$ [cm]", "vtx_matchgvs_d3d", eff_path)

In [67]:
numerators   = [n["vtx_matchgv_sv_d3dsig"], n["vtx_matchgv_svt_d3dsig"], p["vtx_matchgv_sv_d3dsig"], p["vtx_matchgv_svt_d3dsig"]]
denominators = [n["vtx_gv_d3dsig"],         n["vtx_gv_d3dsig"],          p["vtx_gv_d3dsig"],         p["vtx_gv_d3dsig"]]
plotEffs(numerators, denominators, nb["vtx_gv_d3dsig"], labels, "$d_{3D}$ significance", "vtx_matchgv_d3dsig", eff_path)
numerators   = [n["vtx_matchgvs_sv_d3dsig"], n["vtx_matchgvs_svt_d3dsig"], p["vtx_matchgvs_sv_d3dsig"], p["vtx_matchgvs_svt_d3dsig"]]
denominators = [n["vtx_gvs_d3dsig"],         n["vtx_gvs_d3dsig"],          p["vtx_gvs_d3dsig"],         p["vtx_gvs_d3dsig"]]
plotEffs(numerators, denominators, nb["vtx_gvs_d3dsig"], labels, "$d_{3D}$ significance", "vtx_matchgvs_d3dsig", eff_path)

In [68]:
numerators   = [n["vtx_matchgv_sv_motherPdgId"], n["vtx_matchgv_svt_motherPdgId"], p["vtx_matchgv_sv_motherPdgId"], p["vtx_matchgv_svt_motherPdgId"]]
denominators = [n["vtx_gv_motherPdgId"],         n["vtx_gv_motherPdgId"],          p["vtx_gv_motherPdgId"],         p["vtx_gv_motherPdgId"]]
plotEffs(numerators, denominators, nb["vtx_gv_motherPdgId"], labels, "", "vtx_matchgv_motherPdgId", eff_path, xticks)
numerators   = [n["vtx_matchgvs_sv_motherPdgId"], n["vtx_matchgvs_svt_motherPdgId"], p["vtx_matchgvs_sv_motherPdgId"], p["vtx_matchgvs_svt_motherPdgId"]]
denominators = [n["vtx_gvs_motherPdgId"],         n["vtx_gvs_motherPdgId"],          p["vtx_gvs_motherPdgId"],         p["vtx_gvs_motherPdgId"]]
plotEffs(numerators, denominators, nb["vtx_gvs_motherPdgId"], labels, "", "vtx_matchgvs_motherPdgId", eff_path, xticks)

In [69]:
plot_end = time.time()
print("Plotting runtime was %.2f minutes." % ((plot_end-plot_start)/60))

Plotting runtime was 34.83 minutes.


In [70]:
plotter_end = time.time()
print("Script runtime was %.2f minutes." % ((plotter_end-plotter_start)/60))

Script runtime was 87.40 minutes.


# Plotting script done! Check https://btvweb.web.cern.ch/Offline/Phase2/etsai/ for plots.